# Прогнозирование оттока клиентов

### Описание задачи

Имеются данные об ушедших и оставшихся клиентах банка. По этим данным требуется спрогнозировать, уйдет клиент из банка в ближайшее время или останется. Качество модели машинного обучения оценивается с помощью *F1*-меры и *AUC-ROC*. 

### План работы

1. Загрузка и подготовка данных
    1. Выполнение обзора данных
    1. Выполнение предобработки данных
2. Решение без учета дисбаланса классов
    2. Исследование баланса классов
    2. Обучение модели логистической регрессии
    2. Обучение модели решающего дерева
    2. Обучение модели случайного леса
3. Решение с учетом дисбаланса классов
    3. Устранение дисбаланса и обучение модели логистической регрессии
    3. Устранение дисбаланса и обучение модели решающего дерева
    3. Устранение дисбаланса и обучение модели случайного леса
    3. Сравнение моделей по метрикам *F1* и *AUC-ROC*
4. Тестирование моделей
    4. Тестирование модели логистической регрессии
    4. Тестирование модели решающего дерева
    4. Тестирование модели случайного леса
5. Вывод: сравнение результатов тестирования и выявление лучшей модели.

### Описание данных

#### Признаки

* `RowNumber` — индекс строки
* `CustomerId` — идентификатор клиента
* `Surname` — фамилия
* `CreditScore` — кредитный рейтинг
* `Geography` — страна проживания
* `Gender` — пол
* `Age` — возраст
* `Tenure` — сколько лет человек является клиентом банка
* `Balance` — баланс на счёте
* `NumOfProducts` — количество продуктов банка, используемых клиентом
* `HasCrCard` — наличие кредитной карты
* `IsActiveMember` — активность клиента
* `EstimatedSalary` — предполагаемая зарплата

#### Целевой признак

* `Exited` — факт ухода клиента

## Обзор и подготовка данных

Обзор и подготовка данных будет включать в себя загрузку, заполнение пропусков, масштабирование и кодирование.

In [1]:
#Импорт необходимых библиотек и функций

import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler 

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_auc_score

In [2]:
#Загрузка данных
data = pd.read_csv('C:/Users/datasets/customer_churn.csv')

In [3]:
#Просмотр данных
data

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Для улучшения обучения модели и повышения ее качества удалим из данных признаки, которые могут привести к ухудшению обучения модели и таким образом к ухудшению её качества. К таким признакам может относиться, например, порядковый индекс объекта.

In [4]:
data = data.drop(['RowNumber','Surname', 'CustomerId'],axis=1)

In [5]:
#Проверка данных на пропуски
data.isna().sum()

CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

Пропуски стоят в столбце, в котором указывается сколько лет человек является клиентом банка. Можно предположить, что данные в этом столбце в некоторых случаях не указывались для новых клиентов банка, а значит заполнять их логичнее нулями.

In [6]:
#Заполнение пропусков в данных
data['Tenure'] = data['Tenure'].fillna(0)

In [7]:
#Повторная проверка на пропуски в данных
data.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

Вывод: все пропуски были заполнены.

Преобразование категориальных признаков в численные прямым кодированием (англ. One-Hot Encoding, OHE):

In [8]:
data_ohe = pd.get_dummies(data, drop_first=True)

Разделение признаков на целевые и остальные:

In [9]:
#Целевые признаки
target = data_ohe['Exited']

In [10]:
#Основные признаки
features = data_ohe.drop('Exited', axis=1)

Разбиение данных на обучающую 60%, валидационную 20% и тестовую 20% выборки:

In [11]:
#Разбиение данных на обучающую и валидационную выборки
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)

In [12]:
#Разбиение, в свою очередь, валидационной выборки на валидационную и тестовую
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

Проверка результата разбиения на соответствие пропорциям 60%,20%,20%

In [13]:
#Размер обучающей выборки
target_train.shape

(6000,)

In [14]:
#Размер валидационной выборки
target_valid.shape

(2000,)

In [15]:
#Размер тестовой выборки
target_test.shape

(2000,)

Масштабирование численных признаков:

In [16]:
#Список численных признаков, которые будем масштабировать
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [17]:
#Создание объекта для стандартизации данных
scaler = StandardScaler()

In [18]:
#Настройка объекта стандартизации на обучающих данных
scaler.fit(features_train[numeric])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [19]:
pd.options.mode.chained_assignment = None

In [20]:
#Преобразование обучающей выборки
features_train[numeric] = scaler.transform(features_train[numeric])

In [21]:
#Преобразование валидационной выборки
features_valid[numeric] = scaler.transform(features_valid[numeric])

In [22]:
#Преобразование тестовой выборки
features_test[numeric] = scaler.transform(features_test[numeric])

## Решение без учета дисбаланса классов

Исследование баланса классов, обучение модели без учета дисбаланса, расчет F1-меры

### Исследование баланса классов

In [23]:
#Доли классов в исходных данных
data['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

In [24]:
#Доли классов в обучающей выборке
target_train.value_counts(normalize=True)

0    0.800667
1    0.199333
Name: Exited, dtype: float64

In [25]:
#Доли классов в валидационной выборке
target_valid.value_counts(normalize=True)

0    0.791
1    0.209
Name: Exited, dtype: float64

In [26]:
#Доли классов в тестовой выборке
target_test.value_counts(normalize=True)

0    0.7885
1    0.2115
Name: Exited, dtype: float64

Вывод: рассмотрев несколько выборок мы убедились, что соотношение долей практически одно и то же.

### Логистическая регрессия

Логистическая регрессия. Создадим, обучим и проверим с помощью F1-меры модель логистической регрессии.

In [27]:
#Создание модели логистической регрессии
model_lr = LogisticRegression(random_state=12345, solver='liblinear')

In [28]:
#Обучение модели логистической регрессии
model_lr.fit(features_train, target_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [29]:
#Предсказания модели логистической регрессии
predicted_valid = model_lr.predict(features_valid)

In [30]:
#Вычисление F1-меры для модели логистической регрессии
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.33389544688026984


### Решающее дерево

Решающее дерево. Создадим, обучим и проверим с помощью F1-меры модель решающего дерева.

In [31]:
#Создание модели решающего дерева
model_dtc = DecisionTreeClassifier(random_state=12345, max_depth=9)

In [32]:
#Обучение модели решающего дерева
model_dtc.fit(features_train, target_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=9,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

In [33]:
#Предсказания модели решающего дерева
predicted_valid = model_dtc.predict(features_valid)

In [34]:
#Вычисление F1-меры для модели решающего дерева
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5786516853932585


### Случайный лес

Случайный лес. Создадим, обучим и проверим с помощью F1-меры модель случайного леса.

In [35]:
#Создание модели случайного леса
model_rfc = RandomForestClassifier(random_state=12345, n_estimators=200, max_depth=9)

In [36]:
#Обучение модели случайного леса
model_rfc.fit(features_train, target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=9, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [37]:
#Предсказания модели случайного леса
predicted_valid = model_rfc.predict(features_valid)

In [38]:
#вычисление F1-меры для модели случайного леса
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5705329153605015


### Вывод

Вывод. Баланс классов был исследован на исходных данных и дополнительно на трех выборках - обучающей, валидационной и тестовой. Соотношение долей классов во всех случаях, как можно было предположить, получилось практически одним и тем же. Исследование задачи проводилось с помощью трёх моделей - логистической регрессии, решающего дерева и случайного леса. Каждая модель была обучена на тестовой выборке и проверена на валидационной выборке с помощью F1-меры. 

## Решение с учетом дисбаланса классов

In [39]:
#Функция увеличения выборки для борьбы с дисбалансом классов

def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

### Логистическая регрессия

Логистическая регрессия. Устранение дисбаланса классов в логистической регрессии и проверка с помощью F1-меры. Наилучшим балансом классов считается тот, который дает наилучшее значение F1-меры.

In [40]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

In [41]:
#После увеличения выборки просмотр соотношения долей классов
target_upsampled.value_counts(normalize=True)

1    0.554525
0    0.445475
Name: Exited, dtype: float64

При создании далее модели логистической регрессии использовался параметр class_weight, который "уравновешивает" классы и тем самым также борется с дисбалансом.

In [42]:
#Создание модели логистической регрессии с использование параметра class_weight='balanced'
model_lr = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')

In [43]:
#Обучение модели логистической регрессии на увеличенной выборке с лучше сбалансированными классами
model_lr.fit(features_upsampled, target_upsampled)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='liblinear', tol=0.0001,
                   verbose=0, warm_start=False)

In [44]:
#Предсказания модели логистической регрессии
predicted_valid = model_lr.predict(features_valid)

In [45]:
#Вычисление F1-меры для модели логистической регрессии
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.4888888888888888


AUC-ROC логистической регрессии

In [46]:
#Вычисление вероятностей классов каждого объекта валидационной выборки
probabilities_valid = model_lr.predict_proba(features_valid)

In [47]:
#Вероятность класса 1 у каждого объекта валидационной выборки
probabilities_one_valid = probabilities_valid[:, 1]

In [48]:
#Вычисление значения AUC-ROC для логистической регрессии
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [49]:
#Вывод полученного значения AUC-ROC
auc_roc

0.7634754625905068

### Решающее дерево

Решающее дерево. Устранение дисбаланса классов в случае модели решающего дерева и проверка с помощью F1-меры. Наилучшим балансом классов считается тот, который дает наилучшее значение F1-меры.

In [50]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

In [51]:
#После увеличения выборки просмотр соотношения долей классов
target_upsampled.value_counts(normalize=True)

1    0.554525
0    0.445475
Name: Exited, dtype: float64

In [52]:
#Создание модели решающего дерева с использование параметра class_weight='balanced'
model_dtc = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=5)

In [53]:
#Обучение модели решающего дерева на увеличенной выборке с лучше сбалансированными классами
model_dtc.fit(features_upsampled, target_upsampled)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=5,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=12345, splitter='best')

In [54]:
#Предсказания модели решающего дерева
predicted_valid = model_dtc.predict(features_valid)

In [55]:
#Вычисление F1-меры для модели решающего дерева
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5963791267305644


AUC-ROC решающего дерева

In [56]:
#Вычисление вероятностей классов каждого объекта валидационной выборки
probabilities_valid = model_dtc.predict_proba(features_valid)

In [57]:
#Вероятность класса 1 у каждого объекта валидационной выборки
probabilities_one_valid = probabilities_valid[:, 1]

In [58]:
#Вычисление значения AUC-ROC для модели решающего дерева
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [59]:
#Вывод полученного значения AUC-ROC
auc_roc

0.8310244134068074

### Случайный лес

Случайный лес. Устранение дисбаланса классов в случае модели случайного леса и проверка с помощью F1-меры. Наилучшим балансом классов считается тот, который дает наилучшее значение F1-меры.

In [60]:
#Обращение к функции увеличения выборки для устранения дисбаланса
features_upsampled, target_upsampled = upsample(features_train, target_train, 5)

In [61]:
#После увеличения выборки просмотр соотношения долей классов
target_upsampled.value_counts(normalize=True)

1    0.554525
0    0.445475
Name: Exited, dtype: float64

In [62]:
#Создание модели случайного леса с использование параметра class_weight='balanced'
model_rfc = RandomForestClassifier(random_state=12345, class_weight='balanced', n_estimators=1000, max_depth=11)

In [63]:
#Обучение модели случайного леса на увеличенной выборке с лучше сбалансированными классами
model_rfc.fit(features_upsampled, target_upsampled)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=11, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=None, oob_score=False,
                       random_state=12345, verbose=0, warm_start=False)

In [64]:
#Предсказания модели случайного леса
predicted_valid = model_rfc.predict(features_valid)

In [65]:
#Вычисление F1-меры для модели случайного леса
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6232876712328768


AUC-ROC случайного леса

In [66]:
#Вычисление вероятностей классов каждого объекта валидационной выборки
probabilities_valid = model_rfc.predict_proba(features_valid)

In [67]:
#Вероятность класса 1 у каждого объекта валидационной выборки
probabilities_one_valid = probabilities_valid[:, 1]

In [68]:
#Вычисление значения AUC-ROC для модели случайного леса
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)

In [69]:
#Вывод полученного значения AUC-ROC
auc_roc

0.8517759604159231

### Вывод

Вывод. Поскольку при исследовании баланса классов оказалось, что количество 1 меньше чем 0, для борьбы с дисбалансом была использована функция увеличения выборки, которая увеличивает число объектов со значением целевого признака 1. В результате, после обращения к данной функции мы получаем новую обучающую выборку с большим числом объектов и с лучше сбалансированными 1 и 0. Затем мы обучаем каждую модель на новой обучающей выборке. При создании каждой модели в качестве еще одного способа для борьбы с дисбалансом используется параметр class_weight='balanced'. Далее выполняем предсказания с её помощью на валидационной выборке и вычисляем для неё F1-меру. Сравнивая F1-меру каждой модели можно сказать, что лучшее значение у модели случайного леса, значение F1-меры у неё удалось довести до 0.623. Метрика AUC-ROC у модели случайного леса также обладает наилучшим значением по сравнению с метриками других моделей, её значение равно 0.85. Следовательно, наилучшей моделью является модель случайного леса. Далее осуществим тестирование моделей, проверив их метрики на тестовой выборке.

## Тестирование модели

### Логистическая регрессия

In [70]:
#Предсказания модели логистической регрессии на тестовой выборке
predicted_test = model_lr.predict(features_test)

In [71]:
#Вычисление F1-меры для модели логистической регрессии на тестовой выборке
print("F1:", f1_score(target_test, predicted_test))

F1: 0.4797238999137188


In [72]:
#Вычисление вероятностей классов каждого объекта тестовой выборки
probabilities_test = model_lr.predict_proba(features_test)

In [73]:
#Вероятность класса 1 у каждого объекта тестовой выборки
probabilities_one_test = probabilities_test[:, 1]

In [74]:
#Вычисление значения AUC-ROC для модели логистической регрессии на тестовой выборке
auc_roc = roc_auc_score(target_test, probabilities_one_test)

In [75]:
#Вывод полученного значения AUC-ROC
auc_roc

0.7417456312746319

### Решающее дерево

In [76]:
#Предсказания модели решающего дерева на тестовой выборке
predicted_test = model_dtc.predict(features_test)

In [77]:
#Вычисление F1-меры для модели решающего дерева на тестовой выборке
print("F1:", f1_score(target_test, predicted_test))

F1: 0.5809128630705395


In [78]:
#Вычисление вероятностей классов каждого объекта тестовой выборки
probabilities_test = model_dtc.predict_proba(features_test)

In [79]:
#Вероятность класса 1 у каждого объекта тестовой выборки
probabilities_one_test = probabilities_test[:, 1]

In [80]:
#Вычисление значения AUC-ROC для модели решающего дерева на тестовой выборке
auc_roc = roc_auc_score(target_test, probabilities_one_test)

In [81]:
#Вывод полученного значения AUC-ROC
auc_roc

0.8355347481752318

### Случайный лес

In [82]:
#Предсказания модели случайного леса на тестовой выборке
predicted_test = model_rfc.predict(features_test)

In [83]:
#Вычисление F1-меры для модели случайного леса на тестовой выборке
print("F1:", f1_score(target_test, predicted_test))

F1: 0.6008968609865472


In [84]:
#Вычисление вероятностей классов каждого объекта тестовой выборки
probabilities_test = model_rfc.predict_proba(features_test)

In [85]:
#ероятность класса 1 у каждого объекта тестовой выборки
probabilities_one_test = probabilities_test[:, 1]

In [86]:
#Вычисление значения AUC-ROC для модели случайного леса на тестовой выборке
auc_roc = roc_auc_score(target_test, probabilities_one_test)

In [87]:
#Вывод полученного значения AUC-ROC
auc_roc

0.8595741682669461

### Вывод

Вывод. В результате подсчета метрик F1-мера и AUC-ROC на тестовой выборке наилучшие показатели у модели случайного леса - F1-мера равна 0.60, и AUC-ROC равен 0.85. Тестирование моделей показывает, что модели обучены корректно, поскольку разница метрик незначительная и что наилучшей и с приемлемым качеством моделью является модель случайного леса.